In [1]:
import os
from pathlib import Path
from collections import defaultdict

import numpy as np
import laspy

from sklearn.neighbors import NearestNeighbors

from scipy.spatial import ConvexHull

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import knn_graph, MessagePassing
from torch_geometric.utils import softmax

import context
import utils 
import utils_ChallengeBuilding3D as ut_CB3D


c:\Users\oscar\anaconda3\envs\cvpr\lib\site-packages\torch_geometric\typing.py:72: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
c:\Users\oscar\anaconda3\envs\cvpr\lib\site-packages\torch_geometric\typing.py:99: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(
c:\Users\oscar\anaconda3\envs\cvpr\lib\site-packages\torch_geometric\typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


DEFINE FOLDERS

In [2]:
base_folder = Path(os.getcwd())
folder = base_folder.parents[0] / 'data'
output_folder = base_folder.parents[0] / 'outputs'

print ('Base folder: ', base_folder)
print ('Folder: ', folder)
print ('Output folder:', output_folder)

Base folder:  c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Building_3D_Challenge\script
Folder:  c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Building_3D_Challenge\data
Output folder: c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Building_3D_Challenge\outputs


In [3]:
las_files = ut_CB3D.xyz_to_las(folder)
print(f'Converted .xyz to .las files: {las_files}. Type of las_files: {type(las_files)}')

Converted .xyz to .las files: ['c:\\Users\\oscar\\OneDrive - Fondazione Bruno Kessler\\Building_3D_Challenge\\data\\10052.las', 'c:\\Users\\oscar\\OneDrive - Fondazione Bruno Kessler\\Building_3D_Challenge\\data\\9960.las', 'c:\\Users\\oscar\\OneDrive - Fondazione Bruno Kessler\\Building_3D_Challenge\\data\\9961.las', 'c:\\Users\\oscar\\OneDrive - Fondazione Bruno Kessler\\Building_3D_Challenge\\data\\9962.las', 'c:\\Users\\oscar\\OneDrive - Fondazione Bruno Kessler\\Building_3D_Challenge\\data\\9963.las', 'c:\\Users\\oscar\\OneDrive - Fondazione Bruno Kessler\\Building_3D_Challenge\\data\\9964.las', 'c:\\Users\\oscar\\OneDrive - Fondazione Bruno Kessler\\Building_3D_Challenge\\data\\9965.las', 'c:\\Users\\oscar\\OneDrive - Fondazione Bruno Kessler\\Building_3D_Challenge\\data\\9966.las', 'c:\\Users\\oscar\\OneDrive - Fondazione Bruno Kessler\\Building_3D_Challenge\\data\\9967.las', 'c:\\Users\\oscar\\OneDrive - Fondazione Bruno Kessler\\Building_3D_Challenge\\data\\9968.las', 'c:\\Use

In [4]:
points_dict = ut_CB3D.process_las_files(folder)

In [ ]:
points_dict.items()

In [ ]:
# Initialize dictionaries to store the results for each file
all_points = {}
all_center_mass = {}
all_shifted_points = {}
all_cov_matrix = {}

# Process each set of 3D points in points_dict
for file_name, data in points_dict.items():

    # Retrieve original points
    points = data['points_3d']
    print(f'File name: {file_name}\nOriginal points:\n{points[:5]}... (showing first 5 of {len(points)})')

    # Calculate center of mass and shift points
    center_mass, shifted_points = ut_CB3D.center_data(points)
    print(f'Center of Mass: {center_mass}')
    print(f'Shifted points:\n{shifted_points[:5]}... (showing first 5 of {len(shifted_points)})')

    # Compute covariance matrix of the shifted points
    cov_matrix = ut_CB3D.compute_covariance_matrix(shifted_points)
    print(f'Covariance matrix:\n{cov_matrix}')

    # Register points back by adding the center of mass
    registered_points = ut_CB3D.recenter_data(shifted_points, center_mass)
    print(f'Registered points:\n{registered_points[:5]}... (showing first 5 of {len(registered_points)})')

    # Verify if registered points are equal to the original points
    assert np.allclose(points, registered_points), "Registered points do not match the original points."

    # Store results in the respective dictionaries
    all_points[file_name] = registered_points
    all_center_mass[file_name] = center_mass
    all_shifted_points[file_name] = shifted_points
    all_cov_matrix[file_name] = cov_matrix


In [23]:
#assert np.allclose(points, registered_points), "Registered points do not match the original points."

In [25]:
def partition_and_compute_covariance(closest_points, n):

    dot_products = np.dot(closest_points, n)

    N_i_upper = closest_points[dot_products >= 0]
    N_i_lower = closest_points[dot_products < 0]

    # Check if either partition is empty
    if N_i_upper.size == 0 or N_i_lower.size == 0:
        
        print("Empty partition detected.")
        
        return None, None, None, None, None, None

    # Compute covariance matrices for both partitions
    K_i_upper = np.cov(N_i_upper, rowvar=False)
    K_i_lower = np.cov(N_i_lower, rowvar=False)

    # Perform SVD on the covariance matrices
    _, Sigma_i_upper, _ = np.linalg.svd(K_i_upper)
    _, Sigma_i_lower, _ = np.linalg.svd(K_i_lower)

    # Compute the centers of mass for both partitions
    N_i_upper_center = np.mean(N_i_upper, axis=0)
    N_i_lower_center = np.mean(N_i_lower, axis=0)

    return N_i_upper, N_i_lower, Sigma_i_upper, Sigma_i_lower, N_i_upper_center, N_i_lower_center


In [29]:
# Initialize the results list to collect data
results = []

# Process each set of 3D points in points_dict
for file_name, data in points_dict.items():
    points = data['points_3d']  # Get the 3D points from the dictionary

    # Compute the covariance matrix for the entire set of points
    cov_matrix = ut_CB3D.compute_covariance_matrix(points)

    # Perform SVD
    U, singular_values, Vt = ut_CB3D.perform_svd(cov_matrix)

    # Check if SVD was successful
    if U is None or singular_values is None or Vt is None:
        print(f"SVD failed for file: {file_name}")
        continue

    # Center the points by subtracting the mean
    mean, shifted_points = ut_CB3D.center_data(points)

    # Normalize the shifted points using the singular values and U
    normalized_points = ut_CB3D.normalize_neighborhood(shifted_points, singular_values, U)

    # Extract eigenvector of the smallest singular value and closest points
    n, closest_points = ut_CB3D.extract_eigenvector_and_neighbors(normalized_points, Vt)

    # Values for unpacking edges (assuming partition_and_compute_covariance is defined elsewhere)
    N_i_upper, N_i_lower, Sigma_i_upper, Sigma_i_lower, N_i_upper_center, N_i_lower_center = partition_and_compute_covariance(closest_points, n)

    # Calculate perpendicular and tangential distances using the closest points, mean, and eigenvector n
    s_perpendicular, s_tangential = ut_CB3D.calculate_distances(closest_points, mean, n)

    # Store the results or perform additional processing
    results.append({
        'file_name': file_name,
        'mean': mean,
        'shifted_points': shifted_points,
        'normalized_points': normalized_points,
        'points': points,
        'n': n,
        'closest_points': closest_points,
        'N_i_upper': N_i_upper,
        'N_i_lower': N_i_lower,
        'Sigma_i_upper': Sigma_i_upper,
        'Sigma_i_lower': Sigma_i_lower,
        'N_i_upper_center': N_i_upper_center,
        'N_i_lower_center': N_i_lower_center,
        's_perpendicular': s_perpendicular,
        's_tangential': s_tangential
    })



Empty partition detected.
Empty partition detected.


DEEP LEARNING APPROACH

In [16]:
k = 3 # k >> include more points in the neighborhood, k << focus on a more local neighborhood

In [17]:
class PCEDNet(nn.Module):
    def __init__(self, input_dim=12, hidden_dim=64, output_dim=2, radius=0.7):
        
        super(PCEDNet, self).__init__()
        self.radius = radius  # Radius for edge detection

        # MLP layers to embed features
        self.mlp1 = nn.Linear(input_dim, hidden_dim)
        self.mlp2 = nn.Linear(hidden_dim, hidden_dim)
        self.mlp3 = nn.Linear(hidden_dim, hidden_dim)

        # Batch Normalization
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.bn3 = nn.BatchNorm1d(hidden_dim)

        # Attention layer: computes attention scores for neighbor features
        self.attention = nn.Linear(2 * hidden_dim, 1)

        # Classification layer
        self.fc = nn.Linear(hidden_dim, output_dim)

        # Activation
        self.relu = nn.ReLU()

    def forward(self, points, features):
        B, N, _ = points.shape

        # Step 1: Embed the input features using MLP layers
        x = self.mlp1(features)                       # (B, N, hidden_dim)
        x = x.view(B * N, -1)                         # Reshape to (B*N, hidden_dim) for BatchNorm1d
        x = self.relu(self.bn1(x))                    # (B*N, hidden_dim)

        x = self.mlp2(x)                              # (B*N, hidden_dim)
        x = self.relu(self.bn2(x))                    # (B*N, hidden_dim)

        x = self.mlp3(x)                              # (B*N, hidden_dim)
        x = self.relu(self.bn3(x))                    # (B*N, hidden_dim)

        x = x.view(B, N, -1)                          # Reshape back to (B, N, hidden_dim)

        # Step 2: Compute radius graph
        points_flat = points.view(B * N, 3)  # Flatten points to (B*N, 3)
        edge_index = radius_graph(points_flat, 0.70)  # (2, E)

        #  edge_index = radius_graph(points_flat, r=radius)  # (2, E)

        # Step 3: Message Passing with Attention
        source = x.view(B * N, -1)[edge_index[0]]     # (E, hidden_dim)
        target = x.view(B * N, -1)[edge_index[1]]     # (E, hidden_dim)

        concat = torch.cat([source, target], dim=1)   # (E, 2*hidden_dim)
        attn_scores = self.attention(concat)          # (E, 1)
        attn_scores = F.leaky_relu(attn_scores)
        attn_scores = F.softmax(attn_scores, dim=0)  # Apply softmax over the edge dimension

        weighted = source * attn_scores               # (E, hidden_dim)

        # Aggregate weighted messages back to the nodes
        agg = torch.zeros_like(x.view(B * N, -1))     # (B*N, hidden_dim)
        agg = agg.index_add(0, edge_index[0], weighted)
        agg = agg.view(B, N, -1)                      # (B, N, hidden_dim)

        # Step 4: Classify each point based on the aggregated features
        logits = self.fc(agg)                         # (B, N, output_dim)
        edge_probabilities = F.log_softmax(logits, dim=-1)  # (B, N, output_dim)

        return edge_probabilities

In [19]:
def compute_scale_feature(points_3d, k):
    
    n_points = points_3d.shape[0]
    features = np.zeros((n_points, 12))

    # Use NearestNeighbors to find the k nearest neighbors for each point
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(points_3d)
    distances, indices = nbrs.kneighbors(points_3d)
    
    # Calculate the center of mass for each neighborhood
    N_i_k = np.mean(points_3d[indices], axis=1)  # Center of mass for scale k
    
    # Set k0 to the minimum of k0 or the number of points
    k0 = min(max(k, 128), n_points)  # Ensure k0 does not exceed the number of points
    nbrs_k0 = NearestNeighbors(n_neighbors=k0, algorithm='auto').fit(points_3d)
    _, indices_k0 = nbrs_k0.kneighbors(points_3d)
    N_i_k0 = np.mean(points_3d[indices_k0], axis=1)  # Center of mass for scale k0

    # Calculate tangential and perpendicular distances
    for i in range(n_points):
        delta_N = N_i_k[i] - (N_i_k0[i] - points_3d[i])
        ni_k0 = (N_i_k0[i] - points_3d[i]) / np.linalg.norm(N_i_k0[i] - points_3d[i])
        
        # Perpendicular component
        c_i_k_perp = np.dot(delta_N, ni_k0)
        
        # Tangential component
        c_i_k_tang = np.linalg.norm(delta_N - c_i_k_perp * ni_k0)

        # Fill the feature vector for the point
        features[i, :3] = N_i_k[i]         # Center of mass at scale k
        features[i, 3:6] = N_i_k0[i]       # Center of mass at largest scale
        features[i, 6:9] = delta_N         # Difference between centers of mass
        features[i, 9] = c_i_k_perp        # Perpendicular distance
        features[i, 10] = c_i_k_tang       # Tangential distance
        features[i, 11] = distances[i].mean()  # Mean distance to neighbors

    return features

In [21]:
def features_embedding(points_dict, k_scales=[2, 4, 8, 16]):

    results = {}
    
    for file_name, data in points_dict.items():
        print(f"Processing file: {file_name}")

        points_3d = data.get('points_3d')
        if points_3d is None:
            print(f"Error: Missing 'points_3d' data for {file_name}.")
            continue
        
        points_3d = np.array(points_3d)
        multi_scale_features = []
        
        for k in k_scales:
            scale_features = compute_scale_feature(points_3d, k)
            multi_scale_features.append(scale_features)

        # Concatenate features from all scales
        X_i = np.hstack(multi_scale_features)
        results[file_name] = X_i

        print(f"Completed processing for file: {file_name}")

    return results

In [23]:
# Parameters
batch_size = 2
k = 10
radius = 0.70

In [24]:
def decimate_list(num_points_list):
    return num_points_list[::2]

# Sample num_points_list (you should replace this with your actual data)
num_points_list = [100, 150, 200, 250]  # Example list of number of points
num_points_list = decimate_list(num_points_list)

print(num_points_list)

[100, 200]


In [25]:
points_batch = []
features_batch = []
labels_batch = []

# Find maximum number of points in the batch
max_num_points = max(num_points_list)

for i in range(batch_size):
    num_points = num_points_list[i]  # Get the number of points for this batch item
    
    # Generate random points (e.g., points on a sphere)
    theta = np.random.uniform(0, 2 * np.pi, num_points)
    phi = np.random.uniform(0, np.pi, num_points)
    
    x = radius * np.sin(phi) * np.cos(theta)
    y = radius * np.sin(phi) * np.sin(theta)
    z = radius * np.cos(phi)
    points = np.stack([x, y, z], axis=1)  # (N, 3)
    
    # Compute scale features
    features = compute_scale_feature(points, k = 10)
    
    # Generate synthetic labels (for demonstration, randomly assign edges)
    labels = np.random.randint(0, 2, size=(num_points,))  # (N,)
    
    # Pad points, features, and labels
    pad_size_points = max_num_points - num_points
    points_padded = np.pad(points, ((0, pad_size_points), (0, 0)), mode='constant', constant_values=0)
    
    pad_size_features = max_num_points - features.shape[0]
    features_padded = np.pad(features, ((0, pad_size_features), (0, 0)), mode='constant', constant_values=0)
    
    pad_size_labels = max_num_points - len(labels)
    labels_padded = np.pad(labels, (0, pad_size_labels), mode='constant', constant_values=0)
    
    points_batch.append(points_padded)
    features_batch.append(features_padded)
    labels_batch.append(labels_padded)

# Convert lists to numpy arrays for easier manipulation
points_batch = np.array(points_batch)  # Shape: (batch_size, max_num_points, 3)
features_batch = np.array(features_batch)  # Shape: (batch_size, max_num_points, 12)
labels_batch = np.array(labels_batch)  # Shape: (batch_size, max_num_points)


In [26]:
model = PCEDNet(input_dim = 12, hidden_dim = 64, output_dim = 2, radius = 0.7)

In [27]:
def extract_top_edges(edge_probabilities, top_k=10):

    # We need to flatten the tensor to sort the edges
    edge_probabilities = edge_probabilities.view(-1)  # Flatten to (B * N, output_dim)
    
    # Get the indices of the top_k probabilities
    topk_values, topk_indices = torch.topk(edge_probabilities, top_k, largest = True)
    
    return topk_indices.tolist(), topk_values.tolist()

In [28]:
def process_data_and_extract_edges(points_dict, model, k=10, top_k=10):

    results = []

    for file_name, data in points_dict.items():
        points = data['points_3d']  # Get the 3D points from the dictionary

        # Compute features
        features = compute_scale_feature(points, k)

        # Convert to PyTorch tensors
        points_tensor = torch.tensor(points, dtype=torch.float).unsqueeze(0)  # (1, N, 3)
        features_tensor = torch.tensor(features, dtype=torch.float).unsqueeze(0)  # (1, N, 12)

        # Forward pass to obtain edge probabilities
        edge_probabilities = model(points_tensor, features_tensor)  # (B, N, output_dim)

        # Assume edge_probabilities shape is (B, N, N) where N is the number of points
        edge_probabilities = edge_probabilities.squeeze(0)  # Remove batch dimension -> (N, N)
        N = edge_probabilities.shape[0]

        # Generate all possible point pairs (i, j) for the edges
        edges = [(i, j) for i in range(N) for j in range(i+1, N)]

        # Flatten the edge probabilities and map them to the corresponding point pairs
        edge_probabilities_flat = edge_probabilities.view(-1)
        
        # Extract top_k edge probabilities and their indices
        topk_values, topk_indices = torch.topk(edge_probabilities_flat, top_k, largest=True)

        # Map the flattened indices back to the corresponding point pairs (edges)
        top_edges = [edges[idx] for idx in topk_indices.tolist()]

        # Print debugging information
        print(f'File {file_name} Edge probability values {topk_values.tolist()}')
        print(f'File {file_name} Top edges: {top_edges}')

        # Store the results for the file, including the edges and probabilities
        results.append({
            'file_name': file_name,
            'points': points.tolist(),  # Store the points for later use if needed
            'top_edges': top_edges,  # Actual point pairs (edges)
            'top_edges_probabilities': topk_values.tolist()  # Edge probabilities
        })

    return results


In [29]:
from scipy.spatial import cKDTree

def radius_graph(points, radius):

    # points_np = points.detach().cpu().numpy()  # Convert to NumPy array
    points_np = points
    tree = cKDTree(points_np)  # Create a KDTree for efficient radius queries

    edge_index_list = []

    for i in range(len(points_np)):
        indices = tree.query_ball_point(points_np[i], radius)  # Find neighbors within the radius
        for idx in indices:
            if idx != i:  # Avoid self-loops
                edge_index_list.append((i, idx))

    # Convert edge_index_list to tensor
    edge_index = torch.tensor(edge_index_list, dtype=torch.long).t().contiguous()
    
    return edge_index

NEURAL NETWORK

In [33]:
edge_indexes = radius_graph(points, radius)

In [34]:
def point_cloud_to_graph(point_cloud, num_neighbors=8):
    
    """ Convert point cloud to graph by finding k nearest neighbors for each point. """
    # Point cloud: (N, 3) -> list of 3D points
    nbrs = NearestNeighbors(n_neighbors=num_neighbors, algorithm='ball_tree').fit(point_cloud)
    edges = nbrs.kneighbors_graph(point_cloud).toarray()  # Adjacency matrix
    
    # Edge index list (2, E) with source and destination node indices for each edge
    edge_index = torch.tensor(edges.nonzero(), dtype=torch.long)
    
    # Convert to PyTorch Geometric Data object
    data = Data(x=torch.tensor(point_cloud, dtype=torch.float), edge_index=edge_index)
    
    return data

In [35]:
list_of_graphs = []

# Iterate through the dictionary and extract point cloud data
for file_name, point_cloud in points_dict.items():
    
    # Extract 3D points from the point_cloud dictionary
    point_cloud_data = point_cloud['points_3d']  # Use 'points_3d' key
    point_cloud_data = np.array(point_cloud_data)
    
    # Ensure point_cloud_data is a 2D array (N, 3)
    if point_cloud_data.shape[1] != 3:
        raise ValueError(f"Point cloud from {file_name} does not have 3 coordinates per point.")
    
    # Convert point cloud to graph
    graph_data = point_cloud_to_graph(point_cloud_data, num_neighbors=8)
    list_of_graphs.append(graph_data)

C:\Users\oscar\AppData\Local\Temp\ipykernel_22468\2467106094.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\b\abs_6fueooay2f\croot\pytorch-select_1707342446212\work\torch\csrc\utils\tensor_new.cpp:278.)
  edge_index = torch.tensor(edges.nonzero(), dtype=torch.long)


In [38]:
from torch_geometric.nn import knn_graph, GCNConv

class PCENet(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(PCENet, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        # data contains point features (x) and edges (edge_index)
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        
        return x

In [39]:
class EdgePredictor(torch.nn.Module):
    def __init__(self, node_feature_dim):
        super(EdgePredictor, self).__init__()
        self.fc1 = torch.nn.Linear(node_feature_dim * 2, 128)
        self.fc2 = torch.nn.Linear(128, 1)

    def forward(self, node_i, node_j):
        # Concatenate features of two nodes
        edge_features = torch.cat([node_i, node_j], dim=-1)
        edge_features = F.relu(self.fc1(edge_features))
        edge_score = torch.sigmoid(self.fc2(edge_features))
        
        return edge_score

OBJ GT

In [40]:
base_folder_obj = Path(os.getcwd()).parents[0] / 'data' / 'tallinn'
folder_obj = base_folder_obj /'train' / 'wireframe' 
output_folder_obj = base_folder_obj / 'outputs'

print ('Base folder: ', base_folder_obj)
print ('Folder obj: ', folder_obj)
print ('Output folder obj:', output_folder_obj)

Base folder:  c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Building_3D_Challenge\data\tallinn
Folder obj:  c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Building_3D_Challenge\data\tallinn\train\wireframe
Output folder obj: c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Building_3D_Challenge\data\tallinn\outputs


In [41]:
obj_files = [f for f in os.listdir(folder_obj) if f.endswith('.obj')]
print(f'Number of .obj files {len(obj_files)} Type of file {type(obj_files)}')


Number of .obj files 7768 Type of file <class 'list'>


In [ ]:
def parse_obj_file(filename):

    vertices = []
    edges = set()  # Use a set to avoid duplicate edges

    with open(filename, 'r') as file:
        for line in file:
            parts = line.split()

            if len(parts) == 0:
                continue
            
            if parts[0] == 'v':  # Vertex
                try:
                    # Ensure there are at least 3 components for the vertex coordinates
                    vertices.append(list(map(float, parts[1:4])))
                except ValueError:
                    print(f"Warning: Invalid vertex data in line: {line.strip()}")

            elif parts[0] == 'l':  # Line
                try:
                    line_indices = [int(p) - 1 for p in parts[1:]]
                    if len(line_indices) == 2:
                        edge = (min(line_indices[0], line_indices[1]), max(line_indices[0], line_indices[1]))
                        edges.add(edge)
                    else:
                        print(f"Warning: Line with incorrect number of indices in line: {line.strip()}")
                except (ValueError, IndexError):
                    print(f"Warning: Invalid line data in line: {line.strip()}")

    vertices = np.array(vertices)
    edges = list(edges)  # Convert the set of edges to a list before returning

    return vertices, edges

# Directory containing .obj files
folder_obj = Path(os.getcwd()).parents[0] / 'data' / 'tallinn' / 'train' / 'wireframe'

# List all .obj files in the directory
obj_files = [f for f in os.listdir(folder_obj) if f.endswith('.obj')]

# Dictionary to store vertices and edges for each .obj file
obj_data = {}

for obj_file in obj_files:
    file_obj_path = os.path.join(folder_obj, obj_file)
    print(f"Parsing file: {file_obj_path}")
    
    # Parse the .obj file and get vertices and edges
    vertices, edges = parse_obj_file(file_obj_path)
    
    # Ensure vertices is a NumPy array
    vertices = np.array(vertices)  # Convert vertices to a NumPy array if not already
    
    if not isinstance(edges, list):
        print(f"Error: edges for {obj_file} is not a list.")
        continue
    
    # Convert edge indices to a list of tuples if not already
    edge_index = [(int(edge[0]), int(edge[1])) for edge in edges]
    
    # Check if vertices has the correct shape
    if len(vertices.shape) != 2 or vertices.shape[1] != 3:
        print(f"Error: vertices for {obj_file} is not a 2D array with (n, 3) shape.")
        continue
    
    # Check if edges is a list of index pairs
    if not all(isinstance(edge, (list, tuple)) and len(edge) == 2 for edge in edges):
        print(f"Error: edges for {obj_file} contains invalid elements.")
        continue
    
    # Generate edge_coordinates from vertices
    edge_coordinates = [(vertices[edge[0]], vertices[edge[1]]) for edge in edges]

    # Store vertices, edge index (indices), and edge coordinates in the dictionary
    obj_data[obj_file] = {
        'vertices': vertices.tolist(),  # Convert NumPy array to list for storage
        'edge_index': edge_index,  # Original index pairs
        'edges': edge_coordinates  # Coordinates of edges (pair of vertex coordinates)
    }
    
    # Output information for verification
    print(f"File: {obj_file}")
    print(f"Vertices shape: {vertices.shape}, Number of edges: {len(edges)}")
    print(f"Sample edge indices (first 5): {edge_index[:5]}")
    print(f"Sample edge coordinates (first 5): {edge_coordinates[:5]}")

In [43]:
def create_ground_truth_tensor(vertices_index, edges):

    ground_truth_tensor = []

    for idx, edge in enumerate(vertices_index):
        edge_data = {
            'index': edge,  # This is the pair of vertex indices
            'coordinates': edges[idx]  # The corresponding coordinates of the edge
        }
        ground_truth_tensor.append(edge_data)

    return ground_truth_tensor


In [ ]:
# Create and store the ground truth tensors and edges
ground_truth_tensors = []
ground_truth_edges = []

for obj_file, data in obj_data.items():
    vertices_index = data['edge_index']
    edges = data['edges']
    
    # Create ground truth tensor
    ground_truth_tensor = create_ground_truth_tensor(vertices_index, edges)
    ground_truth_tensors.append({
        'file': obj_file,
        'tensor': ground_truth_tensor
    })
    
    # Create ground truth edges
    ground_truth_edge_data = [{'index': index, 'coordinates': coordinates} for index, coordinates in zip(vertices_index, edges)]
    ground_truth_edges.append({
        'file': obj_file,
        'edges': ground_truth_edge_data
    })
    
    print(f"Ground Truth Tensor for {obj_file}:")
    print(ground_truth_tensor)

    print(f"Ground Truth Edges for {obj_file}:")
    print(ground_truth_edge_data)

# Optionally, you can print the full list of ground truth tensors and edges
print("All Ground Truth Tensors:")
print(ground_truth_tensors)

print("All Ground Truth Edges:")
print(ground_truth_edges)

In [45]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader


# Example of how to create a Data object
# Replace with your actual data loading process
data_list = []
for obj_file, data in obj_data.items():
    # Create edge_index tensor
    edge_index = torch.tensor(data['edge_index'], dtype=torch.long).t().contiguous()
    
    # Create node features tensor
    x = torch.tensor(data['vertices'], dtype=torch.float)
    
    # Create a Data object
    graph_data = Data(x=x, edge_index=edge_index)
    data_list.append(graph_data)

# DataLoader for point cloud graphs
data_loader = DataLoader(data_list, batch_size=1, shuffle=True)

In [46]:

def prepare_ground_truth_edges(ground_truth_edges):
    # Flatten the list of edge data into a format suitable for loss calculation
    edge_indices = []
    edge_labels = []
    
    for entry in ground_truth_edges:
        for edge_data in entry['edges']:
            edge_indices.append(edge_data['index'])
            edge_labels.append(1)  # Assuming all provided edges are positive examples

    # Convert to tensors
    edge_indices = torch.tensor(edge_indices, dtype=torch.long)
    edge_labels = torch.tensor(edge_labels, dtype=torch.float)

    return edge_indices, edge_labels

In [47]:
import torch
import torch_geometric
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class PCENet(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(PCENet, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        # Ensure data is a torch_geometric.data.Data object
        if not isinstance(data, torch_geometric.data.Data):
            raise TypeError("Input to the model must be a torch_geometric.data.Data object.")

        # Extract node features and edge index from data
        x, edge_index = data.x, data.edge_index
        
        # Forward pass through the network
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        
        return x

In [48]:
edge_indices, ground_truth_edge_labels = prepare_ground_truth_edges(ground_truth_edges)

# Instantiate models
pc_net = PCENet(input_dim=3, hidden_dim=64, output_dim=64)  # Assume 3D point cloud as input
edge_predictor = EdgePredictor(node_feature_dim=64)

# Optimizer and loss function
optimizer = optim.Adam(list(pc_net.parameters()) + list(edge_predictor.parameters()), lr=0.001)
criterion = torch.nn.BCELoss()

# DataLoader for point cloud graphs (assuming you have a list of Data objects)
data_loader = DataLoader(list_of_graphs, batch_size=1, shuffle=True)  # list_of_graphs contains graph Data objects

In [49]:
num_epochs = 50

In [50]:
def compute_hull_polygons(points_3d):
    # Ensure points_3d is a numpy array
    points_3d = np.array(points_3d)
    
    # Compute the convex hull
    hull = ConvexHull(points_3d)
    hull_polygons = []

    # Extract vertices of each face of the convex hull
    for simplex in hull.simplices:
        # Ensure the polygon is closed by repeating the first vertex
        simplex_closed = np.append(simplex, simplex[0])
        polygon = points_3d[simplex_closed]
        hull_polygons.append(polygon)
    
    return hull_polygons, hull



In [51]:
hull_polygons, hull = compute_hull_polygons(points)


COMPUTE EDGES

In [128]:
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def plot_closed_polyline(ax, closed_polyline):
    # Create a list of edges from the ordered vertices
    polyline_edges = [[closed_polyline[i], closed_polyline[i+1]] for i in range(len(closed_polyline) - 1)]
    
    # Create the Line3DCollection for the polyline
    polyline_collection = Line3DCollection(polyline_edges, colors='g', linewidths=2, linestyle='--')
    
    # Add the polyline to the 3D plot
    ax.add_collection3d(polyline_collection)

In [275]:
def compute_perimeter_edges(points_3d, hull, k, max_edges=33):
    features = compute_scale_feature(points_3d, k)
    
    perimeter_edges = set()
    
    for simplex in hull.simplices:
        for i in range(len(simplex)):
            edge = (simplex[i], simplex[(i + 1) % len(simplex)])
            edge = tuple(sorted(edge))
            perimeter_edges.add(edge)
    
    edge_scores = []

    for edge in perimeter_edges:
        p1, p2 = edge
        
        if not (0 <= p1 < len(points_3d) and 0 <= p2 < len(points_3d)):
            continue

        closest_points = np.vstack([points_3d[p1], points_3d[p2]])
        n = points_3d[p2] - points_3d[p1]
        n = n / np.linalg.norm(n)

        try:
            N_i_upper_edge, N_i_lower_edge, Sigma_i_upper_edge, Sigma_i_lower_edge, N_i_upper_center_edge, N_i_lower_center_edge = partition_and_compute_covariance(closest_points, n)
            N_i_upper_full, N_i_lower_full, Sigma_i_upper_full, Sigma_i_lower_full, N_i_upper_center_full, N_i_lower_center_full = partition_and_compute_covariance(points_3d, n)
        except Exception as e:
            print(f"Error in partition_and_compute_covariance for edge {edge}: {e}")
            continue

        Sigma_i_upper = np.mean([Sigma_i_upper_edge, Sigma_i_upper_full], axis=0) if Sigma_i_upper_edge is not None and Sigma_i_upper_full is not None else None
        Sigma_i_lower = np.mean([Sigma_i_lower_edge, Sigma_i_lower_full], axis=0) if Sigma_i_lower_edge is not None and Sigma_i_lower_full is not None else None
        N_i_upper_center = np.mean([N_i_upper_center_edge, N_i_upper_center_full], axis=0) if N_i_upper_center_edge is not None and N_i_upper_center_full is not None else None
        N_i_lower_center = np.mean([N_i_lower_center_edge, N_i_lower_center_full], axis=0) if N_i_lower_center_edge is not None and N_i_lower_center_full is not None else None

        dist_upper_to_edge = np.linalg.norm(N_i_upper_center - np.mean(closest_points, axis=0)) if N_i_upper_center is not None else 0
        dist_lower_to_edge = np.linalg.norm(N_i_lower_center - np.mean(closest_points, axis=0)) if N_i_lower_center is not None else 0

        eigenvalue_ratio_upper = Sigma_i_upper[0] / Sigma_i_upper[-1] if Sigma_i_upper is not None and Sigma_i_upper[-1] > 0 else 0
        eigenvalue_ratio_lower = Sigma_i_lower[0] / Sigma_i_lower[-1] if Sigma_i_lower is not None and Sigma_i_lower[-1] > 0 else 0

        point_density = len(closest_points) / np.linalg.norm(points_3d[p1] - points_3d[p2])

        alpha, beta, gamma = 0.8, 0.15, 0.05
        score = (alpha * (eigenvalue_ratio_upper + eigenvalue_ratio_lower) 
                - beta * (dist_upper_to_edge + dist_lower_to_edge) 
                + gamma * point_density)
        
        edge_scores.append((score, edge))
    
    edge_scores.sort(key=lambda x: x[0], reverse=True)
    top_edges = [edge for score, edge in edge_scores[:max_edges]]
    
    return top_edges


In [276]:
def remove_noise(points_3d, radius=0.05, min_neighbors=5):
    """
    Remove noise from a point cloud based on a distance threshold.

    Parameters:
    - points_3d: Array of shape (n_points, 3) containing the 3D coordinates of the points.
    - radius: Radius for neighbor search (default: 0.05).
    - min_neighbors: Minimum number of neighbors required for a point to be considered non-noise (default: 5).

    Returns:
    - Array of shape (n_filtered_points, 3) with noise points removed.
    """
    # Initialize NearestNeighbors
    nbrs = NearestNeighbors(n_neighbors=min_neighbors, radius=radius).fit(points_3d)
    
    # Find neighbors
    distances, indices = nbrs.radius_neighbors(points_3d)
    
    # Count the number of neighbors for each point
    num_neighbors = np.array([len(idx) for idx in indices])
    
    # Filter points: keep points with at least min_neighbors
    filtered_points = points_3d[num_neighbors >= min_neighbors]
    
    return filtered_points

In [321]:
def plot_point_cloud_and_hull_3figures(file_name, points_3d, hull_polygons, perimeter_edges_1, top_edges):

    fig = plt.figure(figsize=(18, 15))

    # Define subplots
    ax1 = fig.add_subplot(131, projection='3d')
    ax2 = fig.add_subplot(132, projection='3d')
    ax3 = fig.add_subplot(133, projection='3d')

    # Subplot 1: Point cloud only (no hull)
    ax1.scatter(points_3d[:, 0], points_3d[:, 1], points_3d[:, 2], color='r', marker='o', s=0.50, label='Point cloud')
    ax1.set_title(f'POINT CLOUD FOR {file_name}', fontsize=10)
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_zlabel('z')
    ax1.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
    ax1.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
    ax1.zaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
    ax1.tick_params(axis='both', which='major', labelsize=6)
    ax1.tick_params(axis='both', which='minor', labelsize=6)

    # Subplot 2: Point cloud and convex hull polygons (from hull_1, hull_polygons_1)
    ax2.scatter(points_3d[:, 0], points_3d[:, 1], points_3d[:, 2], color='r', marker='o', s=0.50, label='Point cloud')

    for polygon in hull_polygons_1:
        poly = Poly3DCollection([polygon], alpha=0.10, linewidths=0.50, edgecolors='b')
        ax2.add_collection3d(poly)
        
    ax2.set_title(f'POINT CLOUD AND CONVEX HULL (ORIGINAL) FOR {file_name}', fontsize=8)
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    ax2.set_zlabel('z')
    ax2.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
    ax2.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
    ax2.zaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
    ax2.tick_params(axis='both', which='major', labelsize=6)
    ax2.tick_params(axis='both', which='minor', labelsize=6)

    # Subplot 3: Filtered point cloud, convex hull polygons (hull_2), and perimeter edges (perimeter_edges_2)
    ax3.scatter(filtered_points[:, 0], filtered_points[:, 1], filtered_points[:, 2], color='r', marker='o', s=0.50, label='Filtered point cloud')

    # Plot convex hull polygons (from hull_polygons_2)
    for polygon in hull_polygons_2:
        poly = Poly3DCollection([polygon], alpha=0.10, linewidths=0.50, edgecolors='g', linestyle='--')
        ax3.add_collection3d(poly)

    # Compute the convex hull of the filtered points
    hull = ConvexHull(filtered_points)

    # Extract edges (simplices) from the convex hull, which represent the boundary edges
    hull_edges = hull.simplices  # simplices are pairs of indices that form edges

    # Select up to 10 extreme (border) edges from the convex hull
    edges_to_plot_perimeter = [
        [filtered_points[edge[0]], filtered_points[edge[1]]]
        for edge in hull_edges[:20]  # Select up to 10 edges from the convex hull
    ]

    # Plot the selected edges
    if edges_to_plot_perimeter:
        edge_collection_perimeter = Line3DCollection(edges_to_plot_perimeter, colors='r', linewidths=2, alpha=1.0, label='Perimeter Edges')
        ax3.add_collection3d(edge_collection_perimeter)

    # Plot the selected edges
    if edges_to_plot_perimeter:
        edge_collection_perimeter = Line3DCollection(edges_to_plot_perimeter, colors='r', linewidths=2, alpha=1.0, label='Perimeter Edges')
        ax3.add_collection3d(edge_collection_perimeter)

    ax3.set_title(f'FILTERED POINT CLOUD, HULL, PERIMETER EDGES, AND TOP EDGES FOR {file_name}', fontsize=10)
    ax3.set_xlabel('x')
    ax3.set_ylabel('y')
    ax3.set_zlabel('z')
    ax3.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
    ax3.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
    ax3.zaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
    ax3.tick_params(axis='both', which='major', labelsize=6)
    ax3.tick_params(axis='both', which='minor', labelsize=6)

    plt.show()